# Pulling and Cleaning US Wage Data

## Packages, Libraries, and Working Directory
Below are the tools used in the analysis. If a user is looking to run everything themselves, change the file path below. `

In [1]:
import glob ### group together files
import os ### manage file path
import pandas as pd ### dataframe management and manipulation
import numpy as np ### numeric functions such as averages and sums
from urllib import urlretrieve ### accessing BLS datasets from the website
from time import time ### clock to time sections of the code
from datetime import datetime ### getting years other date parts
import zipfile ### unzipping the BLS datasets
from shutil import rmtree ### clearing folders after being merged

### plots show in Jupyter Notebook
%matplotlib inline 

### Adjust the below path per use case
path = "YOUR/FILE/PATH/HERE"
os.chdir(path)

### Check if there is an existing data folder to house downloads
if not os.path.exists("data"):
    os.makedirs("data")
    print("Made folder for downloads")

## Data Mining and Cleaning
Data from this project has been pulled from the BLS site. The first file pulled is a comprehensive list of MSAs, their counties, and the codes the unique codes to identify them. Wage and wage statistics are downloaded next. Each zip folder has about 4 Gb of data and 4,200 CSV files and there is a zip file for each year. Processing the data is out of the scope of this project, so we will focus on data from 2005 onward and only in the construction industry.

Data has already been pre-loaded and cleaned for the purposes of this project.

### Download MSA area definitions
The area_definitions file defines which counties are in every MSA.  Additionally, the below removes US terretories (except Puerto Rico) and US military junctions.

In [3]:
t0 = time()

### check for file in data folder
if not os.path.exists("data/area_definitions_m2016.xlsx"): ### check id file exists
    print("Downloading area definitions")
    url_msaDef = "https://www.bls.gov/oes/current/area_definitions_m2016.xlsx" ### file URL
    area_def = urlretrieve(url_msaDef, "data/area_definitions_m2016.xlsx") ### download the file into data repository
    print("Downloaded in %s seconds" % round(time()-t0, 1))
    
    ### Cleaning the MSA definitions file
    df_fips = pd.read_excel("data/area_definitions_m2016.xlsx",
                        converters={
                            "FIPS code": str,
                            "MSA code (including MSA divisions)": str,
                            "County name (or Township name for the New England states)":str, 
                            "County code": str,
                            "Township code": str,
                            "MSA code for MSAs with divisions": str,
                            "MSA name for MSAs with divisions": str
                        })
    os.remove("data/area_definitions_m2016.xlsx")
    df_fips.rename(columns={"MSA code (including MSA divisions)":"MSA_code"}, inplace=True) ### rename for ease of use
    
    ### Each county is broken into two separate codes, the first is the FIPS code which is a 2-digit identifier for
    ### the state, region, or terretory it is in. The second is the county code, a 3-digit identifer that is unique
    ### to each county in a state. The combiniation of the two codes makes a unique ID for each county at a national
    ### view that's ideal to work with in this analysis
    df_fips["area_fips"] = df_fips["FIPS code"].str.cat(df_fips["County code"].values.astype(str), sep='')
    not_states = ["60", "66", "69", "74", "78"] #US territories and military bases less Puerto Rico
    df_fips = (df_fips[~df_fips["FIPS code"].isin(not_states)])
    
    ### list of other statistical indicators not pertainant to this research
    nonoWords = ["nonmetropolitan", "NECTA"]
    df_fips = (df_fips[~df_fips["MSA name (including MSA divisions)"].str.contains('|'.join(nonoWords))])
    
    ### Columns already used or not needed for this project
    df_fips = df_fips.drop(['FIPS code', 'State', 'State abbreviation', 'County code', 
                            'Township code','MSA code for MSAs with divisions',
                            'MSA name for MSAs with divisions'], axis=1)
    df_fips.rename(columns={"MSA name (including MSA divisions)":"MSA_name",
                            "County name (or Township name for the New England states)":"County_name"},inplace=True)
    
    ### The CSV file from BLS has spaces before each code that needs to be removed
    df_fips['MSA_code'] = df_fips['MSA_code'].str.replace('  ','')
    df_fips['area_fips'] = df_fips['area_fips'].str.replace('  ','')
    
    ### export the results to an Excel file
    df_fips.to_excel("data/area_definitions_m2016.xlsx", sheet_name="1", index=False)
    
else:
    print("Area definitions are already there and cleaned")

Area definitions are already there and cleaned


### Download QCEW zip files
[QCEW](https://www.bls.gov/cew/) is BLS' repository of quarterly wage data that covers 95% of county, MSA and state level wage statistics. The below code pulls the zip file for each year queried (in this project is the prior 12), opens the zip and extracts all the CSVs. Each CSV is the quarterly statistics for each county in the US. 

In [2]:
year = datetime.today().year ### Get this moment's year
print(year)

### Go 12 years back from last year since full year current year will not be published
years = map(str, range(year, year))
print(years)

2017
[]


In [ ]:
for i in years:
    if sum(t.startswith(i + "_summary") for t in os.listdir("data/")) == 1: ### check for year's summary file
        size_sum = float(os.path.getsize("data/" + i + "_summary.xlsx")) ### size of file in bytes
        print(i + " was already processed and is about %.2f MBs" % (size_sum/1000000)) ### size of file in MBs
    elif sum(t.startswith(i + ".q1-q4") for t in os.listdir("data/")) == 1: ### check for year's raw data
        length = len(os.listdir("data/" + i + ".q1-q4.by_area/"))
        size = float(0)
        for  l in os.listdir("data/" + i + ".q1-q4.by_area/"):
            size += os.path.getsize("data/" + i + ".q1-q4.by_area/" + l)
        print(i + " was already downloaded, has %d files, and is about %.2f GBs" % (length,size/1000000000))    
    else: ### download raw data folder for each year
        t1 = time()
        
        for w in os.listdir("data/"):
            if w.startswith(i) == True:
                os.remove("data/" + w) 
            
        print("Downloading " + i + " quarterly data")
        ### dynamic URL for each year
        url = "https://www.bls.gov/cew/data/files/" + i + "/csv/" + i + "_qtrly_by_area.zip"
        urlretrieve(url, "data/" + i + "_qtrly_by_area.zip") ### download contents from the URL

        zip_ref = zipfile.ZipFile("data/" + i + "_qtrly_by_area.zip", 'r') ### read the contents of the folder
        zip_ref.extractall(path + "/data") ### extract the zip file data to the data repository for this project
        zip_ref.close()

        os.remove("data/" + i + "_qtrly_by_area.zip") ### remove the zip folder
        
        length = len(os.listdir("data/" + i + ".q1-q4.by_area/"))
        size = float(0)
        for  l in os.listdir("data/" + i + ".q1-q4.by_area/"):
            size += os.path.getsize("data/" + i + ".q1-q4.by_area/" + l)
        print(i + " downloaded in ~%s seconds, has %d files and is about %.2f GBs"
              % (round(time() - t1, 1), length, size/1000000000))

### Constructing manageable dataframes
BLS pulls together all the counties' data into individual csv files and aggregates them into one zip file by qauarter. We'll use the [glob](https://docs.python.org/2/library/glob.html) package to aggregate all these files (approximately 4,200 per year) into one dataframe that will be easier to work with.

In [ ]:
### Read in the MSA definitions to filter by
df_fips = pd.read_excel("data/area_definitions_m2016.xlsx",
                        index_col=False,
                        converters={'MSA_code': str,
                                   'area_fips': str})

### Industries to filter by
# industries = ["1011", "1012", "1013", "1021", "1022", "1023", "1024", "1025", "1026", "1027"]
industries = ["1012"] ### Construction industry

concatenated_dfs = [] ### initiate an empty list of dataframes for further use
for i in years:  
    if sum(t.startswith(i + "_summary") for t in os.listdir("data/")) == 1: ### check if summary file has been made
        print(i + " dataframe has already been constructed")
        ### if the summary file has already been made, read in that file and append to concatenated_dfs list
        concatenated_dfs.append(pd.read_excel("data/" + i + "_summary.xlsx",
                                              converters={'MSA_code': str,
                                                          'area_fips':str,
                                                          'year': str,
                                                          'industry_code': str,
                                                          'qtr': str}))
    else:      
        print("Building a single dataframe for " + i)

        ### Since we already filtered out the counties and regions we don't need, we'll use the county codes
        ### from the MSA definition file
        fips_list = df_fips["area_fips"]

        t2 = time()
        
        ### join all csv files from raw data zip folder into a list 
        all_files = glob.glob(os.path.join("data/" + i + ".q1-q?.by_area", "*.csv"))
        
        ### Read all csv files from list into Pandas DFs for transformation
        df_from_each_file = (pd.read_csv(f,
                                         index_col=False,
                                         converters={'area_fips': str,
                                                     'own_code': str,
                                                     'industry_code': str},
                                         low_memory=False)
                             for f in all_files)

        ### Melt all the individual data frames into one
        c_df = pd.concat(df_from_each_file, ignore_index=True)

        before = c_df.shape

        ### Drop unused columns or those already in the MSA definitions file
        c_df = c_df.drop(['size_code','disclosure_code','area_title','own_title','industry_title',
                          'agglvl_title','agglvl_code','size_title','lq_disclosure_code','oty_disclosure_code',]
                         ,axis=1)

        ### Get data only from private sector. Government jobs are skewed by prevailing wages
        c_df = c_df[c_df['own_code'] == "5"]
        c_df = c_df.drop(['own_code'], axis=1)

        ### Reduce data to only counties in MSAs 
        c_df = c_df[c_df['area_fips'].isin(fips_list)]

        ### Only work with industries as defined in the list above (for this project it's just construction)
        c_df = c_df[c_df['industry_code'].isin(industries)]

        ### Use counties only in the MSA definitions file
        c_df = pd.merge(df_fips, c_df, on="area_fips", how='right')
        c_df = c_df.drop(['MSA_name', 'County_name'], axis=1)

        after = c_df.shape

        concatenated_dfs.append(c_df) ### append the df from this loop the the main list

        print(i + " data cleaning time is ~%s seconds, dimensions before cleaning were %s, and %s after"
              % (round(time() - t2, 1), before, after))